# Preprocessing

<b>Import Libs</b>

In [1]:
import requests
import datetime
import pandas as pd
import numpy as np
from prophet import Prophet 
import matplotlib.pyplot as plt
 
%matplotlib inline

plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

<b>Get API data from specific date to specific date</b>

In [2]:
def get_historical_data_with_end_data(start_date, finish_date, ticker,
                                      datablock='history', engine='stock', market='shares', board='TQBR',
                                      columns='TRADEDATE, OPEN, CLOSE'):
    """To get historical stocks data from specific till specific date.
    Accept only one ticker. Return list of data: date, open, close.
    :argument start_date: should be in string format like '2023-01-01'
    :argument finish_date: should be in string format like '2023-01-03'
    :argument ticker: accept only one ticker"""

    result = []

    finish_date = datetime.datetime.strptime(finish_date, '%Y-%m-%d').date()
    days = [finish_date - datetime.timedelta(days=i) for i in range(4)]
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()

    def get_data(start_date=start_date, end_date=None):
        URL = f'http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/boards/{board}/securities/{ticker}.json'
        r = requests.get(URL, params={'iss.meta': 'off', 'iss.only': datablock,
                                      'iss.json': 'extended',
                                      f'{datablock}.columns': columns,
                                      'from': start_date,
                                      'till': end_date})

        data = r.json()[1].get(datablock)
        for i in data:
            for key in i:
                result.append(i[key])

        end_date = data[len(data) - 1].get('TRADEDATE')
        end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()
        return end_date

    while True:

        end_date = start_date + datetime.timedelta(days=100)

        if finish_date <= end_date:
            end_date = finish_date

        if end_date in days:  # ?
            if end_date in result:
                return result
            else:
                get_data(start_date=start_date, end_date=end_date)
                return result
        else:
            end_date = get_data(start_date=start_date, end_date=end_date)
            start_date = end_date

In [3]:
def get_historical_data(start_date, ticker,
                        datablock='history', engine='stock', market='shares', board='TQBR',
                        columns='TRADEDATE, OPEN, CLOSE'):
    """To get historical stocks data from specific date. Return list of data: date, open, close.
    :argument start_date: should be in string format like '2023-01-01'"""

    result = []

    date = datetime.datetime.now().date()
    days = [date - datetime.timedelta(days=i) for i in range(4)]

    def recursion(start_date, ticker, finish_date=None):

        if finish_date in days:
            return result
        else:
            URL = f'http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/boards/{board}/securities/{ticker}.json'
            r = requests.get(URL, params={'iss.meta': 'off', 'iss.only': datablock,
                                          'iss.json': 'extended',
                                          f'{datablock}.columns': columns,
                                          'from': start_date})

            data = r.json()[1].get(datablock)
            for i in data:
                for key in i:
                    result.append(i[key])

            finish_date = data[len(data) - 1].get('TRADEDATE')
            finish_date = datetime.datetime.strptime(finish_date, '%Y-%m-%d').date()
            recursion(start_date=finish_date, ticker=ticker, finish_date=finish_date)

    recursion(start_date=start_date, ticker=ticker, finish_date=None)
    return result


<b> Set up ticker for further calculating</b>

In [6]:
ticker = 'MTSS'
finish_date = '2023-03-17'

<b> Get API data by specific ticker and make list of rows</b>

In [7]:
result = get_historical_data_with_end_data(start_date='2018-01-03', finish_date=finish_date, ticker=ticker)

data = []
temporary = []

for i, v in enumerate(result, 1):
    temporary.append(v)
    
    if i % 3 == 0:
        data.append(temporary)
        temporary = []

real_result = get_historical_data(start_date=finish_date, ticker=ticker)

real_data = []
temporary = []

for i, v in enumerate(real_result, 1):
    temporary.append(v)
    
    if i % 3 == 0:
        real_data.append(temporary)
        temporary = []

[['2023-03-17', 248.9, 248.1],
 ['2023-03-20', 248.55, 251.8],
 ['2023-03-21', 251.55, 255.25],
 ['2023-03-22', 256, 254.4],
 ['2023-03-23', 254.4, 254.4],
 ['2023-03-24', 254.5, 252.75],
 ['2023-03-27', 253.1, 256.85],
 ['2023-03-28', 257.25, 257.6],
 ['2023-03-29', 258, 257.95],
 ['2023-03-30', 258, 259.3],
 ['2023-03-31', 259.3, 257.85],
 ['2023-04-03', 258, 258],
 ['2023-04-04', 258, 254.75],
 ['2023-04-05', 255.25, 254.95],
 ['2023-04-06', 255.4, 255.75],
 ['2023-04-07', 256.9, 257.5],
 ['2023-04-10', 258.1, 259.25],
 ['2023-04-11', 259, 257],
 ['2023-04-12', 258.55, 261.25],
 ['2023-04-13', 261.25, 260.4],
 ['2023-04-14', 260.9, 260.05],
 ['2023-04-17', 260.15, 261],
 ['2023-04-18', 262.35, 265.4],
 ['2023-04-19', 265.4, 266.7],
 ['2023-04-20', 267.3, 268.35],
 ['2023-04-21', 269.9, 266.4],
 ['2023-04-24', 266.4, 269.25],
 ['2023-04-25', 269.5, 271.3],
 ['2023-04-26', 271.4, 272.9],
 ['2023-04-27', 272.9, 273.65],
 ['2023-04-28', 274, 272.15],
 ['2023-05-02', 273.75, 270.7],
 ['2

<b>Make DF and preprocessing data</b>

In [10]:
df = pd.DataFrame(columns=['date', 'open', 'close'], data=data)
df.drop(df[df['close'] == 'None'].index, inplace=True)
df = df.reset_index(drop=True)

df = df.rename(columns={'close': 'y', 'date': 'ds'})
df = df.drop(columns=['open'])

df['ds'] = pd.to_datetime(df['ds'])
df['y'] = df['y'].astype('float')

df['y_18'] = df.iloc[:,1].rolling(window=18).mean().round(2)
df['y_50'] = df.iloc[:,1].rolling(window=50).mean().round(2)

display(df.tail())

,ds,y,y_18,y_50
1329,2023-03-13,256.65,253.79,248.18
1330,2023-03-14,255.90,253.98,248.57
1331,2023-03-15,249.95,254.30,248.86
1332,2023-03-16,247.90,254.33,249.09
1333,2023-03-17,248.10,254.26,249.33


In [14]:
df_real = pd.DataFrame(columns=['date', 'open', 'close'], data=real_data[1:])
df_real.drop(df_real[df_real['close'] == 'None'].index, inplace=True)
df_real = df_real.reset_index(drop=True)

df_real = df_real.rename(columns={'close': 'y', 'date': 'ds'})
df_real = df_real.drop(columns=['open'])

df_real['ds'] = pd.to_datetime(df_real['ds'])
df_real['y'] = df_real['y'].astype('float')

display(df_real.head())

#надо соединить две DF в одну, чтобы смотреть факт рядом с прогнозом

,ds,y
0,2023-03-20,251.80
1,2023-03-21,255.25
2,2023-03-22,254.40
3,2023-03-23,254.40
4,2023-03-24,252.75


# Let's Start Prophet

<b>Look at the data</b>

In [ ]:
df.set_index('ds').y.plot()
df.set_index('ds').y_18.plot()
df.set_index('ds').y_50.plot()

**Running Prophet**

In [ ]:
model = Prophet()
model.fit(df)

# Added (AM: changed to 30 days [23.06.2023])
future = model.make_future_dataframe(periods=30, freq = 'd')
#future.tail()

#model.fit(df)
forecast = model.predict(future)
fig = model.plot_components(forecast)

In [ ]:
forecast = model.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

<b>Plotting</b>

In [ ]:
model.plot(forecast);

<b>Visualizing Prophet models</b>

In [ ]:
df.set_index('ds', inplace=True)
forecast.set_index('ds', inplace=True)

In [ ]:
viz_df = df.join(forecast[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')

In [ ]:
#for better vision take only 60 last days
viz_df = viz_df.iloc[len(viz_df['yhat'])-60:]

In [ ]:
viz_df[['y', 'yhat','y_18','y_50']].plot()

<b> Saving project data in *xlsx</b>

In [ ]:
#viz_df.to_excel("forecast_results/MTSS_forecast.xlsx")  